In [46]:
# Import necessary libraries
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.models import CoherenceModel

# Preprocessing

In [47]:
# Function to load data from folder path
def load_data(folder_path):
    articles = []
    
	# Iterate through all files in the folder path
    for file in os.listdir(folder_path):
        with open(os.path.join(folder_path, file), "r", encoding="latin") as f:
            content = f.read()
            articles.append(content)
    return articles

In [48]:
# Function to preprocess the loaded data
def preprocess_data(articles):
    cleaned_articles = []
     # Iterate through all articles in the input list   

    for article in articles:
	# Remove excessive whitespace characters and convert to lowercase
        text = re.sub(r"\s+", " ", article)
        text = re.sub(r"[^a-zA-Z\s]", "", text).lower()
	# Tokenize text into words
        tokens = word_tokenize(text)
	# Remove stop words (common words with little meaning such as "the" or "a")
        stop_words = set(stopwords.words("english"))
        custom_stopwords = {"mr", "said", "u"}  # Add any additional stopwords here
        stop_words = stop_words.union(custom_stopwords)
	# Remove any remaining short words and lemmatize 
        tokens = [token for token in tokens if token not in stop_words]
        tokens = [token for token in tokens if len(token) > 1 or token in {"i", "a"}]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        tokens = [token for token in tokens if token != "u"]
        cleaned_articles.append(tokens)
    return cleaned_articles

In [49]:
# Function to create a corpus and dictionary from the preprocessed data
def create_corpus_dictionary(cleaned_articles):
    dictionary = corpora.Dictionary(cleaned_articles)
    corpus = [dictionary.doc2bow(article) for article in cleaned_articles]
    return corpus, dictionary

In [50]:
# Run preprocessing functions
folder_path = "Articles/"
articles = load_data(folder_path)
cleaned_articles = preprocess_data(articles)
corpus, dictionary = create_corpus_dictionary(cleaned_articles)

# Topic Models

In [51]:
# Function to train an LDA model
def train_lda(corpus, dictionary, num_topics, passes=15):
    lda_model = models.LdaModel(corpus=corpus,
                                num_topics=num_topics,
                                id2word=dictionary,
                                passes=passes,
                                random_state=13)
    return lda_model

In [52]:
# Function to print and plot the topics from an LDA model
def plot_topics(lda_model):
    num_topics = lda_model.num_topics
    topic_words = lda_model.print_topics(num_words=10)
    
    for topic in range(num_topics):
        print(f"Topic {topic}:")
        print(topic_words[topic])

In [53]:
# Function to run the LDA training and save the results to a file
def run_and_save_lda(corpus, dictionary, num_topics, passes, model_number, output_file):
    lda_model = train_lda(corpus, dictionary, num_topics, passes)

    # save the 4 models to Models/ folder
    temp_file = 'Models/lda_model_' + str(model_number+1)
    lda_model.save(temp_file)

    # Save the topic words to output files
    topic_words = lda_model.print_topics(num_words=10)
    with open(output_file, "w") as f:
        for topic in range(num_topics):
            f.write(f"Topic {topic}:\n")
            f.write(str(topic_words[topic]) + "\n\n")

In [54]:
# Define different parameters to build the LDA model
# use different topic and pass count and discuss differences in report
params = [{"num_topics": 5, "passes": 10},{"num_topics": 10, "passes": 10},{"num_topics": 5, "passes": 20},{"num_topics": 10, "passes": 20}]


# Iterating through the parameters and building different LDA models
for i, p in enumerate(params):
    output_file = f"output_{i + 1}.txt"
    run_and_save_lda(corpus, dictionary,  p["num_topics"], p["passes"], i, output_file)

# Evaluation

In [55]:
# Load models from Models folder
model_1 = models.ldamodel.LdaModel.load("Models/lda_model_1")
model_2 = models.ldamodel.LdaModel.load("Models/lda_model_2")
model_3 = models.ldamodel.LdaModel.load("Models/lda_model_3")
model_4 = models.ldamodel.LdaModel.load("Models/lda_model_4")
models_list = [model_1, model_2, model_3, model_4]

In [56]:
# Function to print Coherence Score of model
def compute_coherence(lda_model):
    coherence_lda_model = CoherenceModel(model=lda_model, texts=cleaned_articles, dictionary=dictionary, coherence='c_v')
    coherence_number = coherence_lda_model.get_coherence()
    print('Coherence:  ', coherence_number)

In [58]:
model_number = 0

# Compute Perplexity of all 4 models
for m in models_list:
    model_number+=1
    print('Model ', model_number)
    print('Perplexity: ', m.log_perplexity(corpus))
    compute_coherence(m)
    print('\n')

Model  1
Perplexity:  -8.492769328374036


c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\gensim\topic_coherence\indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


Coherence:   nan


Model  2
Perplexity:  -8.498756589638418
Coherence:   nan


Model  3
Perplexity:  -8.483742503821002
Coherence:   nan


Model  4
Perplexity:  -8.49175200324849
Coherence:   nan




# Visualization

In [ ]:
# Enable Python Notebook display of pyLDAvis
pyLDAvis.enable_notebook()


In [ ]:

# Model 1 visualization
vis = pyLDAvis.gensim_models.prepare(model_1, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.022581 -0.020375       1        1  39.726319
0      0.009021 -0.025137       2        1  20.178448
3      0.013598 -0.014382       3        1  19.859596
2      0.026173  0.052869       4        1  13.965158
4     -0.071373  0.007024       5        1   6.270479, topic_info=          Term          Freq         Total Category  logprob  loglift
2079     state  10031.000000  10031.000000  Default  30.0000  30.0000
2206      time   5658.000000   5658.000000  Default  29.0000  29.0000
1418       new   5987.000000   5987.000000  Default  28.0000  28.0000
2251     trump   7420.000000   7420.000000  Default  27.0000  27.0000
2293    united   3300.000000   3300.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
1867     right    176.641332   3230.555468   Topic5  -5.9311  -0.1370
1482  official    172.800844   3281.033858   Topic5  -5.9531  -0.1745
189     attack    173.417115   3589.662740   Topic5  -5.9496  -0.2608
1339  military    155.360193   3240.907572   Topic5  -6.0595  -0.2685
2424     world    145.394990   2044.401331   Topic5  -6.1258   0.1259

[463 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
40022      1  0.960071  abdulrahim
17089      1  0.974308       abedi
17089      2  0.010947       abedi
17089      3  0.005474       abedi
17089      4  0.005474       abedi
...      ...       ...         ...
33848      5  0.875652        zuck
12533      1  0.084375  zuckerberg
12533      2  0.016875  zuckerberg
12533      3  0.033750  zuckerberg
12533      5  0.852192  zuckerberg

[985 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3, 5])

In [ ]:
# Model 2 visualization
vis = pyLDAvis.gensim_models.prepare(model_2, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.110246  0.003362       1        1  54.691810
9     -0.140805  0.017375       2        1  26.872188
6     -0.118399  0.048778       3        1   6.315203
5     -0.103495 -0.005196       4        1   5.880449
2     -0.093444 -0.081310       5        1   5.522012
1      0.017422  0.017084       6        1   0.717657
0      0.116043  0.004736       7        1   0.000170
8      0.128101 -0.014083       8        1   0.000170
7      0.149535  0.004103       9        1   0.000170
3      0.155288  0.005151      10        1   0.000170, topic_info=          Term          Freq         Total Category  logprob  loglift
2079     state  10230.000000  10230.000000  Default  30.0000  30.0000
1119   islamic   6175.000000   6175.000000  Default  29.0000  29.0000
2251     trump   7623.000000   7623.000000  Default  28.0000  28.0000
1418       new   6108.000000   6108.000000  Default  27.0000  27.0000
2206      time   5770.000000   5770.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
1419      news      0.000635   1852.218710  Topic10  -7.9538  -1.6033
102       also      0.000756   3092.403494  Topic10  -7.7800  -1.9422
1338  militant      0.000576   1570.898586  Topic10  -8.0510  -1.5359
459    company      0.000665   2503.922864  Topic10  -7.9083  -1.8593
1230      like      0.000665   2560.674632  Topic10  -7.9083  -1.8817

[799 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
1          1  0.363798       abbas
1          2  0.082681       abbas
1          3  0.512625       abbas
1          4  0.016536       abbas
1          6  0.016536       abbas
...      ...       ...         ...
37762      1  0.983909       zanna
33837      1  0.975663      zarrab
12533      1  0.920201  zuckerberg
12533      2  0.075426  zuckerberg
35866      3  0.991385        zway

[1147 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 7, 6, 3, 2, 1, 9, 8, 4])

In [ ]:
# Model 3 visualization
vis = pyLDAvis.gensim_models.prepare(model_3, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.060854 -0.019450       1        1  53.686884
4     -0.067414  0.025897       2        1  18.444299
1     -0.068578 -0.061568       3        1  14.079148
3     -0.092350  0.051372       4        1  13.789329
2      0.289197  0.003749       5        1   0.000341, topic_info=                 Term          Freq         Total Category  logprob  loglift
2251            trump   7476.000000   7476.000000  Default  30.0000  30.0000
2079            state  10104.000000  10104.000000  Default  29.0000  29.0000
2293           united   3344.000000   3344.000000  Default  28.0000  28.0000
1119          islamic   6094.000000   6094.000000  Default  27.0000  27.0000
40136           gigot   1020.000000   1020.000000  Default  26.0000  26.0000
...               ...           ...           ...      ...      ...      ...
1201             last      0.000226   2198.554409   Topic5  -9.6783  -3.4993
2149           syrian      0.000227   2355.001119   Topic5  -9.6753  -3.5650
47     administration      0.000213   1927.509100   Topic5  -9.7398  -3.4292
2268              two      0.000215   2131.738850   Topic5  -9.7279  -3.5180
1115              isi      0.000215   2748.992760   Topic5  -9.7285  -3.7729

[488 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
0          1  0.312877       abadi
0          2  0.554397       abadi
0          3  0.054891       abadi
0          4  0.082336       abadi
40021      4  0.966488   abdeslams
...      ...       ...         ...
33848      2  0.918495        zuck
12533      1  0.039840  zuckerberg
12533      2  0.948194  zuckerberg
12533      4  0.015936  zuckerberg
45737      2  0.598852      zuylen

[771 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 2, 4, 3])

In [ ]:
# Model 4 visualization
vis = pyLDAvis.gensim_models.prepare(model_4, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.140706  0.015345       1        1  49.148876
3      0.175266 -0.042951       2        1  20.723063
6      0.153049  0.047497       3        1  10.970809
0      0.174250 -0.092738       4        1   6.551234
4      0.142721  0.056983       5        1   6.303331
2      0.138261  0.026510       6        1   6.302006
1     -0.229052 -0.002502       7        1   0.000170
8     -0.231001 -0.002563       8        1   0.000170
5     -0.230759 -0.002746       9        1   0.000170
7     -0.233440 -0.002836      10        1   0.000170, topic_info=          Term          Freq         Total Category  logprob  loglift
2251     trump   7563.000000   7563.000000  Default  30.0000  30.0000
2079     state  10214.000000  10214.000000  Default  29.0000  29.0000
2206      time   5769.000000   5769.000000  Default  28.0000  28.0000
1418       new   6108.000000   6108.000000  Default  27.0000  27.0000
2148     syria   3984.000000   3984.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
2447      year      0.000071   3461.876607  Topic10 -10.1465  -4.4215
1867     right      0.000071   3286.166936  Topic10 -10.1502  -4.3731
1928  security      0.000068   2084.229289  Topic10 -10.1887  -3.9563
1482  official      0.000070   3332.712499  Topic10 -10.1627  -4.3997
2424     world      0.000068   2084.583501  Topic10 -10.1943  -3.9621

[817 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
1          1  0.182431       abbas
1          2  0.066339       abbas
1          4  0.016585       abbas
1          5  0.514123       abbas
1          6  0.215600       abbas
...      ...       ...         ...
33848      6  0.930564        zuck
12533      1  0.039715  zuckerberg
12533      2  0.079430  zuckerberg
12533      6  0.881676  zuckerberg
35866      5  0.991420        zway

[1190 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 4, 7, 1, 5, 3, 2, 9, 6, 8])